# Deteccion de caracteres con TF y keras

## Importo Librerias

In [4]:
#Importo librerias a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import cv2
import random
from PIL import Image, ImageFilter
from IPython.display import Image as ImageDisplay
from IPython.core.display import HTML
from io import BytesIO
import base64
from sklearn.model_selection import train_test_split

## Funciones auxiliares

## Carga del dataset

In [5]:
# Cargar los datos del archivo npz
composed_numpy_dataset = np.load('/mnt/c/tmp/content/ocr_composed_images_dataset.npy', allow_pickle=True)
# Si es un array de diccionarios
images = [item[0] for item in composed_numpy_dataset]  # imagen en escala de grises (224x224)
bboxes  = [item[1] for item in composed_numpy_dataset]  # lista de bboxes

## Funciones de transformación para agumentado de datos

## Definicion del Modelo

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Lista de dispositivos físicos GPU
gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibles:", gpus)

IMG_SIZE = 224  # Tamaño base de entrenamiento
GRID_SIZE = 56  # Tamaño de salida del modelo (por ejemplo, 8x downsample)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1177869149991465654
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10182721536
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4737167594755023941
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-06-18 15:04:24.228635: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1750269864.229533   16810 gpu_device.cc:2019] Created device /device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


## Generador de mapas y dataset

In [7]:
def encode_bboxes(bboxes, image_shape, grid_size):
    h, w = image_shape
    label = np.zeros((grid_size, grid_size, 5), dtype=np.float32)

    for bbox in bboxes:
        x1, y1, x2, y2 = bbox
        xc = (x1 + x2) / 2
        yc = (y1 + y2) / 2
        bw = x2 - x1
        bh = y2 - y1

        grid_x = int(xc * grid_size / w)
        grid_y = int(yc * grid_size / h)

        if 0 <= grid_x < grid_size and 0 <= grid_y < grid_size:
            label[grid_y, grid_x, 0] = 1  # objectness
            label[grid_y, grid_x, 1:] = [
                xc / w, yc / h,
                bw / w, bh / h
            ]
    return label

In [8]:
def preprocess_example(img, bboxes):
    # Asegurarse que la imagen tenga shape (224, 224, 1)
    img = np.expand_dims(img, axis=-1).astype(np.float32) / 255.0
    label = encode_bboxes(bboxes, image_shape=img.shape[:2], grid_size=GRID_SIZE)
    return img, label

# Crear lista de ejemplos
data = [preprocess_example(img, bb) for img, bb in zip(images, bboxes)]

# Separar imágenes y etiquetas
X, y = zip(*data)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
img = X_train[np.random.randint(len(X_train))]
print("Shape:", img.shape)

def train_gen():
    for img, label in zip(X_train, y_train):
        yield img, label

# Generador para val
def val_gen():
    for img, label in zip(X_val, y_val):
        yield img, label



output_signature = (
    tf.TensorSpec(shape=(224, 224, 1), dtype=tf.float32),
    tf.TensorSpec(shape=(56, 56, 5), dtype=tf.float32)
)

train_ds = tf.data.Dataset.from_generator(
    train_gen,
    output_signature=output_signature
).batch(16).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(
    val_gen,
    output_signature=output_signature
).batch(16).prefetch(tf.data.AUTOTUNE)

Shape: (224, 224, 1)


I0000 00:00:1750269870.967966   16810 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [10]:
#Definicion del Modelo
"""def build_model():
    inputs = layers.Input(shape=(None, None, 1))

    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(2)(x)  # 112x112

    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)  # 56x56

    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)  # 28x28

    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    outputs = layers.Conv2D(5, 1, padding='same')(x)  # (None, 28, 28, 5)

    model = models.Model(inputs, outputs)
    return model"""
def build_model():
    inputs = tf.keras.Input(shape=(None, None, 1))

    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D(2)(x)  # 1/2

    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)  # 1/4

    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    # sin maxpool para mantener la resolución 1/4

    output = tf.keras.layers.Conv2D(5, 1, padding='same', activation='sigmoid')(x)
    return tf.keras.Model(inputs, output)

In [11]:
import tensorflow.keras.backend as K

def objectness_accuracy(y_true, y_pred):
    obj_true = y_true[..., 0]
    obj_pred = y_pred[..., 0]
    return tf.keras.metrics.binary_accuracy(obj_true, obj_pred)

def positive_objectness_error(y_true, y_pred):
    # Extraemos objectness ground truth y predicción
    obj_true = y_true[..., 0]
    obj_pred = y_pred[..., 0]

    # Filtramos solo donde el ground truth es positivo (1)
    mask = tf.equal(obj_true, 1.0)

    # Predicciones en esas posiciones (sin binarizar, puede ser entre 0 y 1)
    preds_pos = tf.boolean_mask(obj_pred, mask)

    # Error absoluto promedio en los positivos (puede cambiarse por otra métrica)
    error = tf.reduce_mean(tf.abs(1.0 - preds_pos))

    return error

In [12]:
def detection_loss(y_true, y_pred):
    obj_true = y_true[..., 0]  # shape: (batch, 28, 28)
    obj_pred = y_pred[..., 0]

    bbox_true = y_true[..., 1:]  # shape: (batch, 28, 28, 4)
    bbox_pred = y_pred[..., 1:]

    # Calculamos la pérdida por pixel, asegurando que mantenga la forma [batch, 28, 28]
    bce = tf.keras.backend.binary_crossentropy(obj_true, obj_pred)
    bce = tf.reshape(bce, tf.shape(obj_true))  # ahora sí, obj_true es [batch, 28, 28]

    # Pérdida de bounding boxes solo donde hay objetos
    mask = tf.expand_dims(obj_true, axis=-1)
    bbox_loss = tf.reduce_sum(mask * tf.square(bbox_true - bbox_pred), axis=-1)

    # Sumamos las dos pérdidas
    total_loss = bce + bbox_loss
    return tf.reduce_mean(total_loss)

In [13]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/garzamorada/notebooks/checkpoints/mejor_modelo.keras',
    monitor='val_objectness_accuracy',  # nombre de la métrica
    mode='max',
    save_best_only=True,
    save_weights_only=False
)

In [14]:
model = build_model()
model.compile(
    optimizer='adam',
    loss=detection_loss,
    metrics=[positive_objectness_error]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    callbacks=[checkpoint_cb]
)

Epoch 1/40


I0000 00:00:1750269877.267905   16892 service.cc:152] XLA service 0x78583006ab80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750269877.267995   16892 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-06-18 15:04:37.366856: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750269878.921047   16892 cuda_dnn.cc:529] Loaded cuDNN version 90501


      1/Unknown 10s 10s/step - loss: 0.6896 - positive_objectness_error: 0.5042

I0000 00:00:1750269885.309450   16892 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   1000/Unknown 28s 18ms/step - loss: 0.0508 - positive_objectness_error: 0.9868

...

2025-06-18 15:18:10.980916: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4712318957794055207
2025-06-18 15:18:10.980987: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1262815417791578386


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0025 - positive_objectness_error: 0.3850

2025-06-18 15:18:28.934113: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4712318957794055207
2025-06-18 15:18:28.934186: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1262815417791578386


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 0.0025 - positive_objectness_error: 0.3850 - val_loss: 0.0035 - val_positive_objectness_error: 0.4309


2025-06-18 15:18:30.751648: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4712318957794055207
2025-06-18 15:18:30.751733: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1262815417791578386


In [43]:
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_44 (MaxPooling2D) │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 28, 28, 5)      │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 279,953 (1.07 MB)

 Trainable params: 93,317 (364.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 186,636 (729.05 KB)

## Cargo el modelo

In [ ]:
from tensorflow.keras.models import load_model
import gradio as gr

# Si tenés una métrica personalizada:
model = load_model("/content/drive/MyDrive/tp_integrador/detect_model_final.keras", custom_objects={"objectness_accuracy": objectness_accuracy, "scanner_loss": scanner_loss})

## Interface con gradio

In [ ]:
# 🧠 Función para predecir y dibujar los bounding boxes
def detectar_caracteres(img):
    # Convertir a escala de grises si es necesario
    if img.mode != 'L':
        img = img.convert('L')

    img_arr = np.array(img).astype(np.float32) / 255.0  # Normalizar
    h, w = img_arr.shape
    input_tensor = img_arr[None, ..., None]  # Shape: (1, alto, ancho, 1)

    # 📈 Predecir con el modelo
    pred = model.predict(input_tensor)[0]  # Shape: (grid_h, grid_w, 5)
    grid_h, grid_w, _ = pred.shape

    # 📦 Interpretar la salida
    threshold = 0.8
    bboxes = []
    for i in range(grid_h):
        for j in range(grid_w):
            objectness = pred[i, j, 0]
            if objectness > threshold:
                x_rel, y_rel, w_rel, h_rel = pred[i, j, 1:]

                # Coordenadas relativas dentro de la celda → coordenadas absolutas en la imagen original
                cx = (j + x_rel) / grid_w * w
                cy = (i + y_rel) / grid_h * h
                bw = w_rel * w
                bh = h_rel * h

                x1 = int(cx - bw / 2)
                y1 = int(cy - bh / 2)
                x2 = int(cx + bw / 2)
                y2 = int(cy + bh / 2)
                bboxes.append((x1, y1, x2, y2))

    # 🖼 Dibujar cajas sobre la imagen original
    img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_GRAY2BGR)
    for x1, y1, x2, y2 in bboxes:
        cv2.rectangle(img_bgr, (x1, y1), (x2, y2), (0, 255, 0), 1)

    return Image.fromarray(img_bgr)

# 🚀 Interfaz Gradio
gr.Interface(
    fn=detectar_caracteres,
    inputs=gr.Image(type="pil", label="Subí una imagen (escala de grises o color)"),
    outputs=gr.Image(label="Imagen con detecciones"),
    title="Detección de caracteres",
    description="Este modelo detecta caracteres (sin clasificarlos) y dibuja los bounding boxes."
).launch()